In [5]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [ ]:
# EXTRACT: Load the dataset and sample 50k random rows
file_path = r"c:\Users\FRBN6849\Downloads\Projet-ETL-main\Projet-ETL-main\data\raw\raw\recipe_sample.csv"
df = pd.read_csv(file_path)
print(f"Original dataset: {len(df)} rows, {len(df.columns)} columns")

df_sample = df.sample(random_state=42)
print(f"Sample extracted: {len(df_sample)} rows")

Original dataset: 20000 rows, 7 columns
Sample extracted: 1 rows


In [7]:
# TRANSFORM: Check for null values
print("Null values per column:")
print(df_sample.isnull().sum())
print(f"\nTotal rows with at least one null: {df_sample.isnull().any(axis=1).sum()}")

Null values per column:
Unnamed: 0     0
title          0
ingredients    0
directions     0
link           0
source         0
NER            0
dtype: int64

Total rows with at least one null: 0


In [8]:
# TRANSFORM: Remove rows with at least one null column
df_no_nulls = df_sample.dropna()
print(f"Rows after removing any nulls: {len(df_no_nulls)}")
print(f"Removed {len(df_sample) - len(df_no_nulls)} rows with null values")

Rows after removing any nulls: 1
Removed 0 rows with null values


In [9]:
# TRANSFORM: Remove duplicates
df_clean = df_no_nulls.drop_duplicates()
print(f"Rows after removing duplicates: {len(df_clean)}")
print(f"Removed {len(df_no_nulls) - len(df_clean)} duplicate rows")

Rows after removing duplicates: 1
Removed 0 duplicate rows


In [10]:
# Show final clean data sample
print("Final clean data sample:")
df_clean.head(10)

Final clean data sample:


,Unnamed: 0,title,ingredients,directions,link,source,NER
10650,16393,Parmesan Chicken(Serves 8),"[""8 chicken breasts, skinned, boned and pounde...","[""Dip each breast in a combination of eggs and...",www.cookbooks.com/Recipe-Details.aspx?id=84167,Gathered,"[""chicken breasts"", ""eggs"", ""milk"", ""bread cru..."
